# Sugges_ #

One of the strategies to improve user experience is to provide user with hints, or, otherwise, to autocomplete his queries. Let's consider suggest.

Today we will practice generating suggestions using [Trie](https://en.wikipedia.org/wiki/Trie) datastructure (prefix tree), see the example below.

Plan:

1. Build Trie based on real search query data provided by AOL company;
2. Generate suggestion based on trie;
3. Measure suggestion speed;
4. Add spellcheck to suggest (optional).


![image](https://www.ritambhara.in/wp-content/uploads/2017/05/Screen-Shot-2017-05-01-at-4.01.38-PM.png)

## Install Trie DS support

You are free to use any library implementation of Trie, as well as the one we suggest.

https://github.com/google/pygtrie

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install pygtrie

  Created wheel for pygtrie: filename=pygtrie-2.3.2-cp36-none-any.whl size=18867 sha256=c31366aeedcd151d4a5fa7b26aee67d6068cbc91576c23e5157035003343321d
  Stored in directory: /root/.cache/pip/wheels/1c/10/3c/2d28c8ac56cda265d0c16ca129f50e5c3526f49a7fbe224cd9
Successfully built pygtrie


In [3]:
import pygtrie
t = pygtrie.CharTrie()
t["this is 1"] = "A"
t["this is 2"] = "B"
t["this is"] = "D"
t["this"] = "D"
t["that is 3"] = "C"

print(t)

n = t.has_node('this') == pygtrie.Trie.HAS_VALUE
s = t.has_node('this') == pygtrie.Trie.HAS_SUBTRIE

print(f"Node = {n}; Subtree = {s}")

for key, val in t.iteritems("this is"):
    print(key, '~', val)

CharTrie(this: D, this is: D, this is 1: A, this is 2: B, that is 3: C)
Node = False; Subtree = False
this is ~ D
this is 1 ~ A
this is 2 ~ B


## 1. Build a trie upon a dataset ##

### 1.1 Read dataset

Download the [dataset](https://drive.google.com/drive/folders/1rOE5eed37Jy2ANQItZVwDIFgPmkCoFu6) (we provide only the first part of the original data for simplicity (~3.5 mln queries)).
Explore the data, see readme file. Load the dataset.

In [4]:
#TODO: Read the dataset
import pandas as pd
data_path = '/content/drive/My Drive/InformationRetrieval2020/Suggest/user-ct-test-collection-01.txt'

aol_data = pd.read_csv(data_path, sep='\t')

print("DS size:", aol_data.shape[0])
print("DS head:")
print(aol_data.head())
print("DS tail:")
print(aol_data.tail())

DS size: 3558411
DS head:
   AnonID                        Query            QueryTime  ItemRank ClickURL
0     142               rentdirect.com  2006-03-01 07:17:12       NaN      NaN
1     142  www.prescriptionfortime.com  2006-03-12 12:31:06       NaN      NaN
2     142                   staple.com  2006-03-17 21:19:29       NaN      NaN
3     142                   staple.com  2006-03-17 21:19:45       NaN      NaN
4     142    www.newyorklawyersite.com  2006-03-18 08:02:58       NaN      NaN
DS tail:
           AnonID  ...                   ClickURL
3558406  24968114  ...                        NaN
3558407  24969251  ...                        NaN
3558408  24969374  ...                        NaN
3558409  24969374  ...  http://www.greatseats.com
3558410  24969374  ...                        NaN

[5 rows x 5 columns]


### 1.2 Build Trie

We want suggest function to be non-sensitive to stop words because we don't want to upset the user if he confuses/omits prepositions, for example. Consider "public events in Innopolis" vs "public events at Innopolis" or "public events Innopolis" - they all mean the same.

Build Trie based on the dataset, storing query statistics such as query frequency, urls and ranks in nodes. Some queries may not have associated urls, others may have multiple ranked urls. Think of the way to store this information.

In [47]:
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from gensim.parsing.preprocessing import remove_stopwords
stop_words = set(stopwords.words('english'))

#TODO: build trie based on data
aol_trie = pygtrie.CharTrie()

class TrieValue():
  def __init__(self, query, url, rank):
    self.frequency = 1
    self.query = query
    self.urls = []
    self.add_url(url, rank)
  def add_url(self, url, rank):
    if url == url: # meaning url is non nan
      self.urls.append((url,rank))

for row in aol_data.itertuples():
  query = row[2]
  rank = row[4]
  url = row[5]
  # remove stop words and extra spaces in a query
  if query==query: # if query is not nan
    query = query.lower()
    query_filtered = remove_stopwords(query) # preprocess_query(query)
    if aol_trie.has_key(query_filtered): # if there is a key
      value = aol_trie[query_filtered]
    else:
      value = {}
    if query not in value:
      value[query] = TrieValue(query, url, rank)
    else:
      value[query].frequency += 1
      value[query].add_url(url, rank)
    aol_trie[query_filtered] = value

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [48]:
# test trie
for key, val in aol_trie.iteritems("sample q"):
    print(key, '~', val)

sample question surveys ~ {'sample question surveys': <__main__.TrieValue object at 0x7f945033d6d8>}
sample questions immigration interview ~ {'sample questions for immigration interview': <__main__.TrieValue object at 0x7f9530d18e48>}
sample questions interview ~ {'sample questions for interview': <__main__.TrieValue object at 0x7f9530d182b0>}
sample questions family interview ~ {'sample questions for family interview': <__main__.TrieValue object at 0x7f9530d1abe0>}
sample questions sociology race ethnicity ~ {'sample questions sociology race and ethnicity': <__main__.TrieValue object at 0x7f9530c90160>}
sample questions biology ~ {'sample questions biology': <__main__.TrieValue object at 0x7f9530b482e8>}
sample questions citizenship test ~ {'sample questions for us citizenship test': <__main__.TrieValue object at 0x7f94e2959630>}
sample questionarie teaching evaluation ~ {'sample questionarie teaching evaluation': <__main__.TrieValue object at 0x7f9514531a58>}
sample questionnaire te

## 2. Write a suggest function which is non-sensitive to stop words ##

Suggest options for user query based on Trie you just built.
Output results sorted by frequency, print query count for each suggestion. If there is an url available, print the url too. If multiple url-s are available, print the one with the highest rank (the less the better).

Q: What is the empirical threshold for minimal prefix for suggest?

In [49]:
def complete_user_query(query, trie, top_k=5):
    #TODO: suggest top_k options for a user query
    # sort results by frequency, suggest first ranked urls if available
    # step 1. check if query in the tree, if not return empty suggestion
    query = query.lower()
    query = remove_stopwords(query)
    # step 2. take all nodes from query subtree
    results = []
    try:
      for key, val in aol_trie.iteritems(query):
        for k in val:
          results.append(val[k])
    except KeyError:
      print('Nothing to suggest')

    results = sorted(results, key=lambda x:x.frequency, reverse=True)[:top_k]
    # leave only high rank queries
    for res in results:
      if len(res.urls)==0:
        link = ''
      else:
        # sort links by rank
        links = sorted(res.urls, key=lambda x: x[1])
        link=links[0][0]
      print(f'Count {res.frequency}: {res.query} {link}')

        
inp = "trie"
print("Query:", inp)
print("Results:")
complete_user_query(inp, aol_trie)

Query: trie
Results:
Count 5: tried and true tattoo http://www.triedntruetattoo.com
Count 3: triest 
Count 3: triethanalomine http://avalon.unomaha.edu
Count 2: has anyone ever tried the air chair http://www.tmcowners.com
Count 2: tried and failed 


## 3. Measure suggest speed ##

Check how fast your search is working. Consider changing your code if it takes too long on average.

In [50]:
import time

inp_queries = ["inf", "the best ", "information retrieval", "sherlock hol", "carnegie mell", 
               "babies r", "new york", "googol", "inter", "USA sta", "Barbara "]

#TODO: measure avg execution time per query
times = []
for inp in inp_queries:
  start = time.time()
  print("Query:", inp)
  print("Results:")
  complete_user_query(inp, aol_trie)
  end = time.time()
  times.append(end-start)

print(f'Queries are handled in {np.mean(times)} ms on average')

Query: inf
Results:
Count 94: information clearing house http://www.informationclearinghouse.info
Count 72: information on training puppy http://www.101-dog-training-tips.com
Count 59: inflatable slides 
Count 46: infopass http://infopass.uscis.gov
Count 40: infolanka http://www.infolanka.com
Query: the best 
Results:
Count 685: best buy http://www.bestbuy.com
Count 257: bestcounter.biz 
Count 224: bestbuy http://www.bestbuy.com
Count 158: bestbuy.com http://www.bestbuy.com
Count 99: best western http://www.bestwestern.com
Query: information retrieval
Results:
Nothing to suggest
Query: sherlock hol
Results:
Count 4: sherlock holmes http://www.sherlockian.net
Count 2: sherlock holmes society http://www.realtime.net
Count 2: sherlock holmes chronological order http://www.geocities.com
Count 1: sherlock holmes address 
Count 1: sherlock holmes audiotapes 
Query: carnegie mell
Results:
Count 6: carnegie mellon http://www.cmu.edu
Count 1: carnegie mellon university http://www.cmu.edu
Query:

## 4. Bonus task ##

Add spellchecking to your suggest.

In our test queries basically nothing changes if we add spellcheck.


In [57]:
! pip install pyspellchecker
from spellchecker import SpellChecker

def spellcheck_and_complete_user_query(query, trie, top_k=5):
    #TODO: suggest top_k options for a user query
    # sort results by frequency, suggest first ranked urls if available
    # step 1. check if query in the tree, if not return empty suggestion
    spell = SpellChecker()
    corrected_query = spell.correction(query)
    corrected_query = corrected_query.lower()
    corrected_query = remove_stopwords(corrected_query)

    query = query.lower()
    query = remove_stopwords(query)
    # step 2. take all nodes from query subtree
    results = []
    try:
      for key, val in aol_trie.iteritems(query):
        for k in val:
          results.append(val[k])
    except KeyError:
      pass

    try:
      for key, val in aol_trie.iteritems(corrected_query):
        for k in val:
          results.append(val[k])
    except KeyError:
      pass
    
    if not results:
      print('Nothing to suggest')
      return

    results = sorted(results, key=lambda x:x.frequency, reverse=True)[:top_k]
    # leave only high rank queries
    for res in results:
      if len(res.urls)==0:
        link = ''
      else:
        # sort links by rank
        links = sorted(res.urls, key=lambda x: x[1])
        link=links[0][0]
      print(f'Count {res.frequency}: {res.query} {link}')


inp_queries = ["inf", "the best ", "information retrieval", "sherlock hol", "carnegie mell", 
               "babies r", "new york", "googol", "inter", "USA sta", "Barbara "]

#TODO: measure avg execution time per query
times = []
for inp in inp_queries:
  start = time.time()
  print("Query:", inp)
  print("Results:")
  spellcheck_and_complete_user_query(inp, aol_trie)
  end = time.time()
  times.append(end-start)

print(f'Queries are handled in {np.mean(times)} ms on average')

Query: inf
Results:
Count 94: information clearing house http://www.informationclearinghouse.info
Count 94: information clearing house http://www.informationclearinghouse.info
Count 72: information on training puppy http://www.101-dog-training-tips.com
Count 72: information on training puppy http://www.101-dog-training-tips.com
Count 59: inflatable slides 
Query: the best 
Results:
Count 685: best buy http://www.bestbuy.com
Count 685: best buy http://www.bestbuy.com
Count 257: bestcounter.biz 
Count 257: bestcounter.biz 
Count 224: bestbuy http://www.bestbuy.com
Query: information retrieval
Results:
Nothing to suggest
Query: sherlock hol
Results:
Count 4: sherlock holmes http://www.sherlockian.net
Count 4: sherlock holmes http://www.sherlockian.net
Count 2: sherlock holmes society http://www.realtime.net
Count 2: sherlock holmes chronological order http://www.geocities.com
Count 2: sherlock holmes society http://www.realtime.net
Query: carnegie mell
Results:
Count 6: carnegie mellon ht